In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import pickle
import json
import cv2
import re
import gc

In [2]:
tf.test.is_gpu_available()

True

In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# load data to extract labels
data_dir = '../mmhs150k/'
model_dir = '../fcm_replication/models/' # grab pretrained models from FCM
tweet_dict = json.load(open(data_dir + 'MMHS150K_GT.json', 'r'))

In [4]:
# load pretrained cnn
from tensorflow.keras.models import load_model

lstm = load_model(model_dir + 'lstm.h5')
cnn = load_model(model_dir + 'cnn_weighted.h5')

print(lstm.summary())
print(cnn.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           7565100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               150600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 7,715,851
Trainable params: 150,751
Non-trainable params: 7,565,100
_________________________________________________________________
None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_____________

In [5]:
# create new models that are all but the last layer
from tensorflow.keras.models import Sequential, Model

text_net = Sequential()
for layer in lstm.layers[:-1]: text_net.add(layer)
print(text_net.summary())

img_net = Sequential()
for layer in cnn.layers[:-3]: img_net.add(layer)
print(img_net.summary())

# free up memory from old models:
del lstm
del cnn
gc.collect()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           7565100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               150600    
Total params: 7,715,700
Trainable params: 150,600
Non-trainable params: 7,565,100
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 2048)              268437504 
Total params:

263272

In [6]:
# method for cleaning text like in https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
def hashtag(text):
    hashtag_body = text.group()[1:]
    if hashtag_body.isupper(): return "<hashtag> {} ".format(hashtag_body.lower())
    else: return ' '.join(["<hashtag>"] + [re.sub(r"([A-Z])",r" \1", hashtag_body, flags=re.MULTILINE | re.DOTALL)])

def allcaps(text): return text.group().lower() + ' <allcaps> '    

def clean_tweet_text(t):
    eyes = r'[8:=;]'
    nose = r"['`\-]?"
    
    t = re.sub(r'https?:\/\/\S+\b|www\.(\w+\.)+\S*', '<url>', t)
    t = re.sub(r'@\w+', '<user>', t)
    t = re.sub(r'{}{}[)dD]+|[)dD]+{}{}'.format(eyes, nose, nose, eyes), '<smile>', t)
    t = re.sub(r'{}{}p+".format(eyes, nose)', '<lolface>', t)
    t = re.sub(r'{}{}\(+|\)+{}{}'.format(eyes, nose, nose, eyes), '<sadface>', t)
    t = re.sub(r'{}{}[\/|l*]'.format(eyes, nose), '<neutralface>', t)
    t = re.sub(r'/', ' / ', t)
    t = re.sub(r'<3','<heart>', t)
    t = re.sub(r'[-+]?[.\d]*[\d]+[:,.\d]*', '<number>', t)
    t = re.sub(r'#\S+', hashtag, t)
    t = re.sub(r'([!?.]){2,}', r'\1 <repeat>', t)
    t = re.sub(r'\b(\S*?)(.)\2{2,}\b', r'\1\2 <elong>', t)
    t = re.sub(r'([A-Z]){2,}', allcaps, t)
    t = re.sub(r'{}'.format(r'[\".,-;&:]'), ' ', t)
    return t.lower()

In [7]:
# custom data generator to handle multimodal data
from random import randint # for random cropping
from tensorflow.keras.preprocessing.sequence import pad_sequences

class MMSeqDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, splits_path, tweet_dict, tokenizer, pad_len, batch_size=32, dim=(299, 299), n_channels=3, 
                 shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.tweet_dict = tweet_dict
        self.tokenizer = tokenizer
        self.pad_len = pad_len
        
        # build labels list and id list
        self.id_list = open(splits_path, 'r').read().splitlines()
        self.labels = dict()
        for id in self.id_list:
            binary_labels = [1 if n > 0 else 0 for n in tweet_dict[id]['labels']]
            label = 1 if sum(binary_labels)/len(tweet_dict[id]['labels']) > 0.5 else 0
            self.labels[id] = label
        
        # create dictionary for embedded sequences (tuple of text and img_text)
        self.text_dict = self.process_text(self.id_list)
        
        self.on_epoch_end()
        self.classes = [self.labels[self.id_list[i]] for i in self.indexes]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.id_list) / self.batch_size)) + 1 # last batch is partial

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:index*self.batch_size + self.batch_size]
        
        
        # Find list of IDs
        id_list_temp = [self.id_list[k] for k in indexes]

        # Generate data
        X_txt, X_img, y = self.__data_generation(id_list_temp)
        
        return [X_txt, X_img], y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.id_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, id_list_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X_img = np.empty((len(id_list_temp), *self.dim, self.n_channels))
        X_txt = np.empty((len(id_list_temp), self.pad_len))
        y = np.empty(len(id_list_temp), dtype=int)

        # Generate data
        for i, ID in enumerate(id_list_temp):
            X_img[i,] = self.process_img(data_dir + 'img_resized/' + ID + '.jpg')
            X_txt[i,] = self.text_dict[ID][0]

            # Store class
            y[i] = self.labels[ID]

        return X_txt, X_img, y
    
    def process_img(self, path): # method for getting image
        img = Image.open(path)
        img.load()
        data = np.asarray(img, dtype='uint8')
        im = self.augment(data)
        
        if im.shape==(self.dim[0], self.dim[1]): im = np.stack((im,)*3, axis=-1) # handle grayscale
        
        return im
    
    def augment(self, im): # random crop and random mirror
        
        # random crop
        x_max, y_max = im.shape[0], im.shape[1]
        x_start, y_start = randint(0, x_max - self.dim[0]), randint(0, y_max - self.dim[1])
        im = im[x_start:x_start + self.dim[0], y_start:y_start + self.dim[1]]
        
        # random mirror
        if randint(0,1): im = np.flip(im, axis=1)
        
        return im
    
    def process_text(self, id_list): # return 
        
        # matrix for texts
        texts = [clean_tweet_text(tweet_dict[ID]['tweet_text']) for ID in id_list]
        sequences = self.tokenizer.texts_to_sequences(texts)
        text_seqs = pad_sequences(sequences, maxlen=self.pad_len)
        
        id_to_seq = dict() # map id to text sequence compatible with embedding layer
        for ID, txt in zip(id_list, text_seqs): id_to_seq[ID] = txt
        
        return id_to_seq

In [8]:
# create data generators
tokenizer = pickle.load(open(model_dir + 'tokenizer.pkl', 'rb'))
pad_len = 50

train_gen = MMSeqDataGenerator(splits_path=data_dir + 'splits/train_ids.txt',
                          tweet_dict=tweet_dict,
                          tokenizer=tokenizer,
                          pad_len=pad_len,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

val_gen = MMSeqDataGenerator(splits_path=data_dir + 'splits/val_ids.txt',
                          tweet_dict=tweet_dict,
                          tokenizer=tokenizer,
                          pad_len=pad_len,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=True)

test_gen = MMSeqDataGenerator(splits_path=data_dir + 'splits/test_ids.txt',
                          tweet_dict=tweet_dict,
                          tokenizer=tokenizer,
                          pad_len=pad_len,
                          batch_size=32,
                          dim=(299, 299),
                          n_channels=3,
                          shuffle=False)

In [9]:
# build same embedding matrix as in the lstm file
from tensorflow.keras.layers import Dense, Input, Embedding, Conv1D

EMBEDDING_DIM = 100
word_index = word_index = tokenizer.word_index
MAX_SEQ_LEN = pad_len

# map word to embedding
embeddings_index = {}
for line in open(os.path.join('../glove', 'glove.twitter.27B.100d.txt')):
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')

# create embedding matrix (words without embeddings get zero embeddings)
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

token_embedding = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQ_LEN,
                            trainable=False)

In [10]:
# build attention model with image vector as query
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention
from tensorflow.keras.layers import concatenate, Attention, Reshape
from tensorflow.keras.layers import Dense, Input, Embedding, Conv1D
from tensorflow.keras.optimizers import Adam

#inputs
text_input = Input((text_net.layers[0].input_shape[-1],)) # get rid of None's in front, context/value
# img_text_input = Input((text_net.layers[0].input_shape[-1],))
img_input = Input((img_net.layers[0].input_shape[1:])) # get rid of None's in front, query

# get [batch_size, Tq, dim] embeddings from text
txt_value_embeddings = token_embedding(text_input)
cnn_layer = Conv1D(filters=100, kernel_size=4) # 1D conv for getting seq from txt
txt_value_seq_encoding = cnn_layer(txt_value_embeddings)

# get [batch_size, Tv, dim] embeddings from image
img_embed = img_net(img_input) # this is the query
img_query_seq_encoding = Dense(100)(img_embed) # linear activation to get same dimension

# pass image text through lstm
# img_text_embed = text_net(img_text_input)

# do attention input to DNN as in link
query_value_attention_seq = tf.keras.layers.Attention()(
    [img_query_seq_encoding, txt_value_seq_encoding])
query_encoding = img_query_seq_encoding
query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)
input_layer = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])

x = Dense(2048, activation='relu')(input_layer)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[text_input, img_input], outputs=prediction)
print(model.summary())

optimizer = Adam(lr = 1e-6)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 2048)         290240288   input_2[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 100)      7565100     input_1[0][0]                    
______________________________________________________________________________________________

In [11]:
# train model
from tensorflow.keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint(model_dir + 'best_att.h5', save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

history = model.fit_generator(train_gen,
                    validation_data=val_gen,
                    shuffle=True,
                    epochs=3,
                    callbacks=[mcp_save])

Epoch 1/3
1206/4214 [=======>......................] - ETA: 1:19:14 - loss: 0.5200 - accuracy: 0.7814

KeyboardInterrupt: 

In [45]:
# test
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import math

y_test = test_gen.classes

# get AUROC
preds = model.predict_generator(test_gen)
print('Test AUROC:', roc_auc_score(y_test, preds))

# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.5] = 1
preds_bin[preds<=0.5] = 0
print('Test Accuracy:', accuracy_score(y_test, preds_bin))

# get F1
print('Test F1:', f1_score(y_test, preds_bin, zero_division=1))
print('Test Precision:', precision_score(y_test, preds_bin, zero_division=1))
print('Test Recall:', recall_score(y_test, preds_bin, zero_division=1))

Test AUROC: 0.57832572
Test Accuracy: 0.5009
Test F1: 0.006370694803902051
Test Precision: 0.6956521739130435
Test Recall: 0.0032


In [46]:
# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.2] = 1
preds_bin[preds<=0.2] = 0
print('Test Accuracy:', accuracy_score(y_test, preds_bin))

# get F1
print('Test F1:', f1_score(y_test, preds_bin, zero_division=1))
print('Test Precision:', precision_score(y_test, preds_bin, zero_division=1))
print('Test Recall:', recall_score(y_test, preds_bin, zero_division=1))

Test Accuracy: 0.5499
Test F1: 0.6089827121883415
Test Precision: 0.5383197665489172
Test Recall: 0.701
